# 

In [7]:
import random
import sys
from collections import deque

In [8]:
# --- Configuration ---
NUM_COMMANDS = 1_000_000
INITIAL_PRICE = 10000
TICK_SIZE = 1
SPREAD_HALFWIDTH = 5

# Probabilities
PROB_PASSIVE_ADD = 0.50  # passive orders are resting 
PROB_AGGRESSIVE_ADD = 0.30  # aggressive orders will most likely result in a match 
PROB_CANCEL = 0.20 


In [9]:
def generate_orders_stateful(filename):
    """
    Generates input, with some consideration of cancel orders
    """
    print(f"Generating {NUM_COMMANDS} state-aware commands to {filename}...")
    
    mid_price = INITIAL_PRICE
    order_id_counter = 1
    
    # Keep track of active resting orders, so cancel orders are more accurate 
    active_bids = deque()
    active_asks = deque()

    with open(filename, 'w') as f:
        for i in range(NUM_COMMANDS):
            # Choose action based on probability 
            action = random.choices(
                population=['passive', 'aggressive', 'cancel'],
                weights=[PROB_PASSIVE_ADD, PROB_AGGRESSIVE_ADD, PROB_CANCEL],
                k=1
            )[0]

            # Change the mean price of commands over time; asset price moving like a random walk
            mid_price += random.choice([-TICK_SIZE, TICK_SIZE])
            best_bid_price = mid_price - SPREAD_HALFWIDTH
            best_ask_price = mid_price + SPREAD_HALFWIDTH

            if action == 'passive':
                side = random.choice(["BUY", "SELL"])
                quantity = random.randint(1, 100)
                
                current_order_id = order_id_counter
                order_id_counter += 1
                
                if side == "BUY":
                    price = best_bid_price - random.randint(0, 5) * TICK_SIZE
                    active_bids.append(current_order_id)
                else: # SELL
                    price = best_ask_price + random.randint(0, 5) * TICK_SIZE
                    active_asks.append(current_order_id)
                
                f.write(f"ADD {side} {price} {quantity}\n")

            # Orders that will most likely result in a match
            elif action == 'aggressive':
                side = random.choice(["BUY", "SELL"])
                quantity = random.randint(1, 100)
                
                current_order_id = order_id_counter
                order_id_counter += 1

                # for an aggressive buy, there must be existing resting ask orders
                if side == "BUY" and active_asks:
                    price = best_ask_price + random.randint(0, 3) * TICK_SIZE
                    f.write(f"ADD {side} {price} {quantity}\n")

                    if active_asks:
                        active_asks.popleft() # Remove the oldest ask
                        
                elif side == "SELL" and active_bids:
                    # Aggressive SELL crosses the spread and likely fills a resting BID
                    price = best_bid_price - random.randint(0, 3) * TICK_SIZE
                    f.write(f"ADD {side} {price} {quantity}\n")
                    
                    if active_bids:
                        active_bids.popleft() # Remove the oldest bid
                else:
                    # Skip iteration if no opposing orders 
                    continue

            elif action == 'cancel':
                if not active_bids and not active_asks:
                    continue # Nothing to cancel

                # Choose a side to cancel from, prioritizing the larger queue
                cancel_side = "BUY"
                # if no buy orders || (sell order exists and a "toss a coin that is heads")
                if not active_bids or (active_asks and random.random() < 0.5):
                    cancel_side = "SELL"
                
                if cancel_side == "BUY" and active_bids:
                    id_to_cancel = active_bids.popleft() # Cancel the oldest bid
                    f.write(f"CANCEL {id_to_cancel}\n")
                elif cancel_side == "SELL" and active_asks:
                    id_to_cancel = active_asks.popleft() # Cancel the oldest ask
                    f.write(f"CANCEL {id_to_cancel}\n")

    print("Generation complete.")



In [6]:
# Main file
if __name__ == "__main__":
    if len(sys.argv) > 1:
        output_filename = sys.argv[1]
    else:
        output_filename = "input_generate_test.txt" # New default name
    generate_orders_stateful(output_filename)

Generating 1000000 state-aware commands to -f...
Generation complete.
